In [2]:
#Connexion drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip "/content/drive/MyDrive/Colab Notebooks/debut_pySpark/StockEtablissement_utf8.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/debut_pySpark/StockEtablissement_utf8.zip
  inflating: StockEtablissement_utf8.csv  


In [6]:
#Configuration de spark
#1 - Téléchargement de JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [16]:
#2 - Installation de Apache Spark 3.0.1 avec Hadoop 2.7 
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [18]:
#3 - Decompression dossier spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz


In [19]:
#4 - installation de la bibliothèque findspark
#Il localisera Spark sur le système et l'importera en tant que bibliothèque standard.
!pip install -q findspark

In [24]:
#5 - chemin de l'environnement
#Cela nous permettra d'exécuter Pyspark dans l'environnement Colab.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [25]:
#Test de la bonne installation et configuration de notre environnement Spark et Hadoop
import findspark
findspark.init()

In [26]:
#Emplacement où Spark est installé
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2'

In [27]:
#Creation d'une SparkSession, qui est le point d'entrée de Spark.
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [28]:
spark

In [29]:
#URL publique pour la page de l'interface utilisateur
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2022-04-26 22:06:53--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.60MB/s    in 2.0s    

2022-04-26 22:06:56 (6.60 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


In [30]:
#Chargement jeu de données
df = spark.read.csv("StockEtablissement_utf8.csv", header=True, inferSchema=True)

In [31]:
#Affichage du schema des données
df.printSchema()

root
 |-- siren: integer (nullable = true)
 |-- nic: integer (nullable = true)
 |-- siret: long (nullable = true)
 |-- statutDiffusionEtablissement: string (nullable = true)
 |-- dateCreationEtablissement: string (nullable = true)
 |-- trancheEffectifsEtablissement: string (nullable = true)
 |-- anneeEffectifsEtablissement: integer (nullable = true)
 |-- activitePrincipaleRegistreMetiersEtablissement: string (nullable = true)
 |-- dateDernierTraitementEtablissement: timestamp (nullable = true)
 |-- etablissementSiege: boolean (nullable = true)
 |-- nombrePeriodesEtablissement: integer (nullable = true)
 |-- complementAdresseEtablissement: string (nullable = true)
 |-- numeroVoieEtablissement: string (nullable = true)
 |-- indiceRepetitionEtablissement: string (nullable = true)
 |-- typeVoieEtablissement: string (nullable = true)
 |-- libelleVoieEtablissement: string (nullable = true)
 |-- codePostalEtablissement: integer (nullable = true)
 |-- libelleCommuneEtablissement: string (nulla

In [32]:
#Les 5 première lignes du jeu de données
df.show(5)

+------+---+-----------+----------------------------+-------------------------+-----------------------------+---------------------------+----------------------------------------------+----------------------------------+------------------+---------------------------+------------------------------+-----------------------+-----------------------------+---------------------+------------------------+-----------------------+---------------------------+-----------------------------------+---------------------------------+------------------------+----------------------+-------------------------+-----------------------------+--------------------------------+-------------------------------+------------------------+------------------------------+----------------------+-------------------------+------------------------+----------------------------+------------------------------------+----------------------------------+-------------------------+-----------------------+--------------------------+---

In [34]:
#nombre de ligne du jeu de donnée
df.count()

32648533

In [35]:
#nombre de partitions utilisés pour traiter le jeu de données
df.rdd.getNumPartitions()

43